In [63]:
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation

from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras import backend as K

from sklearn.metrics import log_loss

from keras.datasets import cifar10

In [39]:
def conv2d_bn(x, nb_filter, nb_row, nb_col, border_mode='same', subsample=(1, 1), name=None):
    """
    Utility function to apply conv + BN for Inception V3.
    """
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
        
    bn_axis = 1
#     x = Convolution2D(nb_filter, nb_row, nb_col,
#                       subsample=subsample,
#                       activation='relu',
#                       border_mode=border_mode,
#                       name=conv_name)(x)
    x = Convolution2D(filters=nb_filter, 
                      kernel_size=(nb_row, nb_col),
                      strides=subsample,
                      activation='relu',
                      padding=border_mode,
                      name=conv_name)(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name)(x)
    
    return x

In [42]:
# def inception_v3_model(img_rows, img_cols, channel=1, num_classes=None):
#     """
#     Inception-V3 Model for Keras

#     Model Schema is based on 
#     https://github.com/fchollet/deep-learning-models/blob/master/inception_v3.py

#     ImageNet Pretrained Weights 
#     https://github.com/fchollet/deep-learning-models/releases/download/v0.2/inception_v3_weights_th_dim_ordering_th_kernels.h5

#     Parameters:
#       img_rows, img_cols - resolution of inputs
#       channel - 1 for grayscale, 3 for color 
#       num_classes - number of class labels for our classification task
#     """
#     channel_axis = 1
#     img_input = Input(shape=(img_rows, img_cols, channel))
#     x = conv2d_bn(img_input, 32, 3, 3, subsample=(2, 2), border_mode='valid')
#     x = conv2d_bn(x, 32, 3, 3, border_mode='valid')
#     x = conv2d_bn(x, 64, 3, 3)
#     x = MaxPooling2D((3, 3), strides=(2, 2))(x)

#     x = conv2d_bn(x, 80, 1, 1, border_mode='valid')
#     x = conv2d_bn(x, 192, 3, 3, border_mode='valid')
#     x = MaxPooling2D((3, 3), strides=(2, 2))(x)

#     # mixed 0, 1, 2: 35 x 35 x 256
#     for i in range(3):
#         branch1x1 = conv2d_bn(x, 64, 1, 1)

#         branch5x5 = conv2d_bn(x, 48, 1, 1)
#         branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

#         branch3x3dbl = conv2d_bn(x, 64, 1, 1)
#         branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
#         branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

#         branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
#         branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
#         x = Merge([branch1x1, branch5x5, branch3x3dbl, branch_pool], mode='concat', concat_axis=channel_axis, name='mixed' + str(i))

#     # mixed 3: 17 x 17 x 768
#     branch3x3 = conv2d_bn(x, 384, 3, 3, subsample=(2, 2), border_mode='valid')

#     branch3x3dbl = conv2d_bn(x, 64, 1, 1)
#     branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
#     branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3, subsample=(2, 2), border_mode='valid')

#     branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
#     x = Merge([branch3x3, branch3x3dbl, branch_pool], mode='concat', concat_axis=channel_axis, name='mixed3')

#     # mixed 4: 17 x 17 x 768
#     branch1x1 = conv2d_bn(x, 192, 1, 1)

#     branch7x7 = conv2d_bn(x, 128, 1, 1)
#     branch7x7 = conv2d_bn(branch7x7, 128, 1, 7)
#     branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

#     branch7x7dbl = conv2d_bn(x, 128, 1, 1)
#     branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
#     branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 1, 7)
#     branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
#     branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

#     branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
#     branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
#     x = Merge([branch1x1, branch7x7, branch7x7dbl, branch_pool], mode='concat', concat_axis=channel_axis, name='mixed4')

#     # mixed 5, 6: 17 x 17 x 768
#     for i in range(2):
#         branch1x1 = conv2d_bn(x, 192, 1, 1)

#         branch7x7 = conv2d_bn(x, 160, 1, 1)
#         branch7x7 = conv2d_bn(branch7x7, 160, 1, 7)
#         branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

#         branch7x7dbl = conv2d_bn(x, 160, 1, 1)
#         branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
#         branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 1, 7)
#         branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
#         branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

#         branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
#         branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
#         x = Merge([branch1x1, branch7x7, branch7x7dbl, branch_pool], mode='concat', concat_axis=channel_axis, name='mixed' + str(5 + i))

#     # mixed 7: 17 x 17 x 768
#     branch1x1 = conv2d_bn(x, 192, 1, 1)

#     branch7x7 = conv2d_bn(x, 192, 1, 1)
#     branch7x7 = conv2d_bn(branch7x7, 192, 1, 7)
#     branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

#     branch7x7dbl = conv2d_bn(x, 160, 1, 1)
#     branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
#     branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)
#     branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
#     branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

#     branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
#     branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
#     x = Merge([branch1x1, branch7x7, branch7x7dbl, branch_pool], mode='concat', concat_axis=channel_axis, name='mixed7')

#     # mixed 8: 8 x 8 x 1280
#     branch3x3 = conv2d_bn(x, 192, 1, 1)
#     branch3x3 = conv2d_bn(branch3x3, 320, 3, 3, subsample=(2, 2), border_mode='valid')

#     branch7x7x3 = conv2d_bn(x, 192, 1, 1)
#     branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7)
#     branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1)
#     branch7x7x3 = conv2d_bn(branch7x7x3, 192, 3, 3, subsample=(2, 2), border_mode='valid')

#     branch_pool = AveragePooling2D((3, 3), strides=(2, 2))(x)
#     x = Merge([branch3x3, branch7x7x3, branch_pool], mode='concat', concat_axis=channel_axis, name='mixed8')

#     # mixed 9: 8 x 8 x 2048
#     for i in range(2):
#         branch1x1 = conv2d_bn(x, 320, 1, 1)

#         branch3x3 = conv2d_bn(x, 384, 1, 1)
#         branch3x3_1 = conv2d_bn(branch3x3, 384, 1, 3)
#         branch3x3_2 = conv2d_bn(branch3x3, 384, 3, 1)
#         branch3x3 = Merge([branch3x3_1, branch3x3_2], mode='concat', concat_axis=channel_axis, name='mixed9_' + str(i))

#         branch3x3dbl = conv2d_bn(x, 448, 1, 1)
#         branch3x3dbl = conv2d_bn(branch3x3dbl, 384, 3, 3)
#         branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 384, 1, 3)
#         branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 384, 3, 1)
#         branch3x3dbl = Merge([branch3x3dbl_1, branch3x3dbl_2], mode='concat', concat_axis=channel_axis)

#         branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
#         branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
#         x = Merge([branch1x1, branch3x3, branch3x3dbl, branch_pool], mode='concat', concat_axis=channel_axis, name='mixed' + str(9 + i))

#     # Fully Connected Softmax Layer
#     x_fc = AveragePooling2D((8, 8), strides=(8, 8), name='avg_pool')(x)
#     x_fc = Flatten(name='flatten')(x_fc)
#     x_fc = Dense(1000, activation='softmax', name='predictions')(x_fc)

#     # Create model
#     model = Model(img_input, x_fc)

#     # Load ImageNet pre-trained data 
#     model.load_weights('keras_weights/inception_v3_weights_tf_dim_ordering_tf_kernels.h5')

#     # Truncate and replace softmax layer for transfer learning
#     # Cannot use model.layers.pop() since model is not of Sequential() type
#     # The method below works since pre-trained weights are stored in layers but not in the model
#     x_newfc = AveragePooling2D((8, 8), strides=(8, 8), name='avg_pool')(x)
#     x_newfc = Flatten(name='flatten')(x_newfc)
#     x_newfc = Dense(num_classes, activation='softmax', name='predictions')(x_newfc)

#     # Create another model with our customized softmax
#     model = Model(img_input, x_newfc)

#     # Learning rate is changed to 0.001
#     sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
#     model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

#     return model 

In [64]:
def inception_v3_model(img_rows, img_cols, channel=1, num_classes=None):
    """
    Inception-V3 Model for Keras

    Model Schema is based on 
    https://github.com/fchollet/deep-learning-models/blob/master/inception_v3.py

    ImageNet Pretrained Weights 
    https://github.com/fchollet/deep-learning-models/releases/download/v0.2/inception_v3_weights_th_dim_ordering_th_kernels.h5

    Parameters:
      img_rows, img_cols - resolution of inputs
      channel - 1 for grayscale, 3 for color 
      num_classes - number of class labels for our classification task
    """
    channel_axis = 1
    img_input = Input(shape=(img_rows, img_cols, channel))
    x = conv2d_bn(img_input, 32, 3, 3, subsample=(2, 2), border_mode='valid')
    x = conv2d_bn(x, 32, 3, 3, border_mode='valid')
    x = conv2d_bn(x, 64, 3, 3)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv2d_bn(x, 80, 1, 1, border_mode='valid')
    x = conv2d_bn(x, 192, 3, 3, border_mode='valid')
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    # mixed 0, 1, 2: 35 x 35 x 256
    for i in range(3):
        branch1x1 = conv2d_bn(x, 64, 1, 1)

        branch5x5 = conv2d_bn(x, 48, 1, 1)
        branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

        branch3x3dbl = conv2d_bn(x, 64, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

        branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
        branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
        x = merge([branch1x1, branch5x5, branch3x3dbl, branch_pool], mode='concat', concat_axis=channel_axis, name='mixed' + str(i))

    # mixed 3: 17 x 17 x 768
    branch3x3 = conv2d_bn(x, 384, 3, 3, subsample=(2, 2), border_mode='valid')

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3, subsample=(2, 2), border_mode='valid')

    branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = merge([branch3x3, branch3x3dbl, branch_pool], mode='concat', concat_axis=channel_axis, name='mixed3')

    # mixed 4: 17 x 17 x 768
    branch1x1 = conv2d_bn(x, 192, 1, 1)

    branch7x7 = conv2d_bn(x, 128, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 128, 1, 7)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

    branch7x7dbl = conv2d_bn(x, 128, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 1, 7)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = merge([branch1x1, branch7x7, branch7x7dbl, branch_pool], mode='concat', concat_axis=channel_axis, name='mixed4')

    # mixed 5, 6: 17 x 17 x 768
    for i in range(2):
        branch1x1 = conv2d_bn(x, 192, 1, 1)

        branch7x7 = conv2d_bn(x, 160, 1, 1)
        branch7x7 = conv2d_bn(branch7x7, 160, 1, 7)
        branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

        branch7x7dbl = conv2d_bn(x, 160, 1, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 1, 7)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 160, 7, 1)
        branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

        branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = merge([branch1x1, branch7x7, branch7x7dbl, branch_pool], mode='concat', concat_axis=channel_axis, name='mixed' + str(5 + i))

    # mixed 7: 17 x 17 x 768
    branch1x1 = conv2d_bn(x, 192, 1, 1)

    branch7x7 = conv2d_bn(x, 192, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 192, 1, 7)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

    branch7x7dbl = conv2d_bn(x, 160, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = merge([branch1x1, branch7x7, branch7x7dbl, branch_pool], mode='concat', concat_axis=channel_axis, name='mixed7')

    # mixed 8: 8 x 8 x 1280
    branch3x3 = conv2d_bn(x, 192, 1, 1)
    branch3x3 = conv2d_bn(branch3x3, 320, 3, 3, subsample=(2, 2), border_mode='valid')

    branch7x7x3 = conv2d_bn(x, 192, 1, 1)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 3, 3, subsample=(2, 2), border_mode='valid')

    branch_pool = AveragePooling2D((3, 3), strides=(2, 2))(x)
    x = merge([branch3x3, branch7x7x3, branch_pool], mode='concat', concat_axis=channel_axis, name='mixed8')

    # mixed 9: 8 x 8 x 2048
    for i in range(2):
        branch1x1 = conv2d_bn(x, 320, 1, 1)

        branch3x3 = conv2d_bn(x, 384, 1, 1)
        branch3x3_1 = conv2d_bn(branch3x3, 384, 1, 3)
        branch3x3_2 = conv2d_bn(branch3x3, 384, 3, 1)
        branch3x3 = merge([branch3x3_1, branch3x3_2], mode='concat', concat_axis=channel_axis, name='mixed9_' + str(i))

        branch3x3dbl = conv2d_bn(x, 448, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 384, 3, 3)
        branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 384, 1, 3)
        branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 384, 3, 1)
        branch3x3dbl = merge([branch3x3dbl_1, branch3x3dbl_2], mode='concat', concat_axis=channel_axis)

        branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = merge([branch1x1, branch3x3, branch3x3dbl, branch_pool], mode='concat', concat_axis=channel_axis, name='mixed' + str(9 + i))

    # Fully Connected Softmax Layer
    x_fc = AveragePooling2D((8, 8), strides=(8, 8), name='avg_pool')(x)
    x_fc = Flatten(name='flatten')(x_fc)
    x_fc = Dense(1000, activation='softmax', name='predictions')(x_fc)

    # Create model
    model = Model(img_input, x_fc)

    # Load ImageNet pre-trained data 
    model.load_weights('keras_weights/inception_v3_weights_tf_dim_ordering_tf_kernels.h5')

    # Truncate and replace softmax layer for transfer learning
    # Cannot use model.layers.pop() since model is not of Sequential() type
    # The method below works since pre-trained weights are stored in layers but not in the model
    x_newfc = AveragePooling2D((8, 8), strides=(8, 8), name='avg_pool')(x)
    x_newfc = Flatten(name='flatten')(x_newfc)
    x_newfc = Dense(num_classes, activation='softmax', name='predictions')(x_newfc)

    # Create another model with our customized softmax
    model = Model(img_input, x_newfc)

    # Learning rate is changed to 0.001
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

    return model 

In [65]:
# Example to fine-tune on 3000 samples from Cifar10
img_rows, img_cols = 299, 299 # Resolution of inputs
channel = 3
num_classes = 10 
batch_size = 16 
nb_epoch = 10

# Load Cifar10 data. Please implement your own load_data() module for your own dataset
(X_train, Y_train), (X_valid, Y_valid) = cifar10.load_data()

# Load our model
model = inception_v3_model(img_rows, img_cols, channel, num_classes)

# Start Fine-tuning
model.fit(X_train, Y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          shuffle=True,
          verbose=1,
          validation_data=(X_valid, Y_valid),
          )

# Make predictions
predictions_valid = model.predict(X_valid, batch_size=batch_size, verbose=1)

# Cross-entropy loss score
score = log_loss(Y_valid, predictions_valid)

F:\Python\Anaconda3-4.2.0\setup\lib\site-packages\ipykernel\__main__.py:40: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
F:\Python\Anaconda3-4.2.0\setup\lib\site-packages\keras\legacy\layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


ValueError: "concat" mode can only merge layers with matching output shapes except for the concat axis. Layer shapes: [(None, 35, 35, 64), (None, 35, 35, 64), (None, 35, 35, 96), (None, 35, 35, 32)]

In [54]:
from keras.layers import Merge

model_left= Sequential()  
model_left.add(Dense(50, input_shape=(784,)))  
model_left.add(Activation('relu')) 
print(model_left.input,model_left.input_shape)
model_left.summary()   
    
model_right = Sequential()  
model_right.add(Dense(50, input_shape=(784,)))  
model_right.add(Activation('relu'))  
print(model_right.input,model_right.input_shape)
model_right.summary()

model = Sequential()  
model.add(Merge([model_left,model_right], mode='concat', concat_axis=1))  

model.add(Dense(10))  
model.add(Activation('softmax'))  
print(model.input_shape)
model.summary()


Tensor("dense_72_input:0", shape=(?, 784), dtype=float32) (None, 784)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_72 (Dense)             (None, 50)                39250     
_________________________________________________________________
activation_72 (Activation)   (None, 50)                0         
Total params: 39,250
Trainable params: 39,250
Non-trainable params: 0
_________________________________________________________________
Tensor("dense_73_input:0", shape=(?, 784), dtype=float32) (None, 784)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_73 (Dense)             (None, 50)                39250     
_________________________________________________________________
activation_73 (Activation)   (None, 50)                0         
Total params: 39,250
Trainable params: 39,250
Non-trainable para

F:\Python\Anaconda3-4.2.0\setup\lib\site-packages\ipykernel\__main__.py:16: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [59]:
from keras.layers.merge import concatenate,add

input1 = Input(shape=(16,))
x1 = Dense(8, activation='relu')(input1)
input2 = Input(shape=(32,))
x2 = Dense(8, activation='relu')(input2)
# added = add([x1, x2])
added = concatenate([x1,x2],axis=-1)

out = Dense(4)(added)
model = Model(inputs=[input1, input2], outputs=out)

print(model.input, model.input_shape)
print(model.output, model.output_shape)
model.summary()

[<tf.Tensor 'input_16:0' shape=(?, 16) dtype=float32>, <tf.Tensor 'input_17:0' shape=(?, 32) dtype=float32>] [(None, 16), (None, 32)]
Tensor("dense_89/BiasAdd:0", shape=(?, 4), dtype=float32) (None, 4)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 16)           0                                            
__________________________________________________________________________________________________
input_17 (InputLayer)           (None, 32)           0                                            
__________________________________________________________________________________________________
dense_87 (Dense)                (None, 8)            136         input_16[0][0]                   
__________________________________________________________________________________________________
dense_